# Segmenting and Clustering Neighborhoods in Toronto

# Part 1

## Web Scraping the Data
This section of the notebook uses the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M in order to obtain the data that is in the table of postal codes and to transform the dat a into a pandas dataframe. 

In [5]:
! pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=6ad900a0c7bef27fc37d578638b222b8d8589e55f8eaa1f06a1cc37c7854ebee
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [8]:
# importing libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

print("required libraries imported")

required libraries imported


In [185]:
# send the GET request
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
result = requests.get(url).text

# pass the data into beautifulsoup
soup = BeautifulSoup(result, 'lxml')

In [186]:
# create column names for the data frame
column_names = ['PostalCode', 'Borough', 'Neighborhood']

df = pd.DataFrame(columns=column_names)

In [187]:
# append the data from soup object to the data frame
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if len(cells)>0:
        df = df.append({'PostalCode': cells[0].text.rstrip('\n'), 'Borough': cells[1].text.rstrip('\n'), 'Neighborhood': cells[2].text.rstrip('\n')}, ignore_index=True)
                       
df.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [191]:
# Ignore cells with a borough that is Not assigned
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [192]:
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])

In [193]:
# pd.set_option('display.max_rows', df.shape[0]+1)
df1 = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [197]:
df.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M1G,North York,Downsview
freq,1,24,4


In [195]:
df1.shape

(103, 3)

### Seps for data cleaning
* For the above data cleaning step python bs4 package was used to scrape the data from website. 
* Then scraped data was converted in to a pandas data frame. The row of the data removed from the data frame if the Borugh was not assigned.
* some postal codes have more than one neighbourhoods, hence they were combine into a single row with multiple neighbourhoods 